In [36]:
import os
import pandas as pd

In [51]:
def extract_qserver_log(qs_log_path, xrf_path, search_line):
    log_fnames = [f for f in os.listdir(qs_log_path) if 'logging.log' in f]
    
    search_on = False
    next_line = 0
    seen = 0
    scan_name = []
    sample_name = []
    user_comment = []

    for log_fname in log_fnames:
        log_path = os.path.join(qs_log_path, log_fname)
        print(f"processing file: {log_path}")
        with open(log_path, 'r') as f:
            lines = f.readlines()
            for line in lines:
                if xrf_path in line:
                    search_on = True

                elif search_on:
                    if search_line in line:
                        seen += 1
                        if seen == 1:
                            scan_name.append(line.split(' ')[5])
                            next_line = 1

                    elif next_line:
                        a = eval(line.split("Scan parameters: ")[1])
                        sample_name.append(a['samplename'])
                        user_comment.append(a['user_comments'])
                        next_line -= 1
                        if next_line == 0:
                            search_on = False
                            seen = 0
    return scan_name, sample_name, user

In [69]:
xrf_path = 'File path is set to W:2idd/2025-1/Paunesku/flyXRF/'
qs_log_paths = ['/home/beams/USER2IDD/bluesky-mic/qs/.logs', '/home/beams8/USER2IDD/bluesky-mic/.logs']
search_line = '--------------------------------'
scan_name = []
sample_name = []
user_comment = []
# qs_log_path2 = '/home/beams8/USER2IDD/bluesky-mic/.logs'
# log_fnames = [f for f in os.listdir(qs_log_path) if 'logging.log' in f]

for qs_log_path in qs_log_paths:
    scanname, samplename, usercomment = extract_qserver_log(qs_log_path, xrf_path, search_line)
    scan_name += scanname
    sample_name += samplename
    user_comment += usercomment


processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.8
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.6
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.7
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.1
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.3
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.2
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.4
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.5
processing file: /home/beams/USER2IDD/bluesky-mic/qs/.logs/logging.log.9
processing file: /home/beams8/USER2IDD/bluesky-mic/.logs/logging.log.8
processing file: /home/beams8/USER2IDD/bluesky-mic/.logs/logging.log.6
processing file: /home/beams8/USER2IDD/bluesky-mic/.logs/logging.log.7
processing file: /home/beams8/USER2IDD/bluesky-mic/.logs/lo

In [70]:
df = pd.DataFrame({k: eval(k) for k in ['scan_name', 'sample_name', 'user_comment']})

In [72]:
df = df.sort_values('scan_name').reset_index(drop=True)

In [74]:
df.to_csv('qserver_log.csv', index=False)